In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()
import pandas as pd
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# x => [공부 시간, 과외 시간]
x = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]] # 7행 2열
print(x)
print(type(x))

xData = np.array(x)
print(xData)
print(type(xData))

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [3]:
# y => [합격 여부] => 실제값
y = [0, 0, 0, 1, 1, 1, 1] # 1행 7열
# 행수를 맞추기 위해 7행 1열로 shape을 만들어야 한다. => 그러려면 numpy배열로 바꾼 후 reshape()을 써야 한다.
print(y)
print(type(y))

yData = np.array(y)
print(yData)
print(type(yData))

# reshape(): 넘파이에서 데이터는 그대로 유지한채 배열의 형태(차원)을 변경한다.
# 1행 7열 넘파이 배열 => 7행 1열 넘파이 배열
yData = np.array(y).reshape(7, 1)
print(yData)
print(type(yData))


[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[0 0 0 1 1 1 1]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


xData와 yData를 저장할 placeholder를 만든다.

In [4]:
# 데이터 개수를 그때 그때 다르게 하기 위해 placeholder를 지정한다.
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성을 이용해서 대입될 넘파이 배열의 차원을 지정해야 한다.
# [None, 2]이라고 적으면 행이 없다는 게 아니고 행에 몇 행이 와도 상관 없고, 열은 무조건 2개가 와야 한다는 뜻
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder 
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData가 대입될 placeholder

가중치와 y절편을 임의의 값으로 정한다.

In [5]:
# 위의 shape=[None, 2]에서의 2와 uniform([2, 1])의 2는 같아야 한다.
a = tf.Variable(tf.random_uniform([2, 1], dtype=tf.float32)) # 난수를 2행 1열로 발생시킨다.
b = tf.Variable(tf.random_uniform([1], dtype=tf.float32))
sess = tf.Session()
sess.run(tf.global_variables_initializer()) 
# print('a = {}, b = {}'.format(sess.run(a), sess.run(b)))
print('a\n', sess.run(a), sep=' ')
print('b\n', sess.run(b))
print('a[0]: {:f}, a[1]: {:f}, b: {:f}'.format(sess.run(a)[0][0], sess.run(a)[1][0], sess.run(b)[0]))
print('a[0]: %f, a[1]: %f, b: %f' % (sess.run(a)[0], sess.run(a)[1], sess.run(b)))

a
 [[0.9606966 ]
 [0.61551523]]
b
 [0.8349191]
a[0]: 0.960697, a[1]: 0.615515, b: 0.834919
a[0]: 0.960697, a[1]: 0.615515, b: 0.834919


시그모이드 방정식, 오차 함수, 경사 하강

In [6]:
# sigmoid(): tensorflow에서 시그모이드 방정식을 계산한다.
# matmul(): tensorflow에서 행렬의 곱(적)을 계산한다. (데이터 * 가중치를 알아서 계산해준다.)
# Y = 1 / (1 + np.e ** - (a1*x1 + a2*x2 + b))
# 시그모이드 방정식, 예측값
y = tf.sigmoid(tf.matmul(X, a) + b) # X에 들어있는 행 데이터에서 a의 가중치만큼 알아서 돌면서 계산해준다.
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y)) # 오차 함수 
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # 경사 하강

sigmoid() 메소드의 실행 결과(y)가 0.5 이상이면 1을 0.5 미만이면 -1을 리턴시킨다.

In [7]:
sess = tf.Session()
# cast(data, dtype): tensorfloew에서 데이터(data)를 dtype으로 형변환한다.
predict = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32)
print(sess.run(predict)) # [1. 2]
# cast() 메소드는 캐스팅 할 데이터가 boolean 타입이면 True는 1로 False는 0으로 형변환한다.
predict = tf.cast(tf.constant([True, False]), dtype=tf.int32)
print(sess.run(predict)) # [1, 0]
predict = tf.cast([0.5 >= 0.5, 0.5 < 0.5], dtype=tf.int32)
print(sess.run(predict)) # [1, 0] True, False와 동일한 결과가 나온다.

[1 2]
[1 0]
[1 0]


sigmoid() 메소드를 실행한 예상값(0 또는 1)을 계산한다.

In [8]:
predict = tf.cast(y >= 0.5, dtype=tf.float32) # 조건이 60점 이상이면 0.6으로 바꾸면 되지~~~

예측값(predict)과 실제값(Y)이 일치하는 정도(정확도, accurancy)를 계산한다.

In [11]:
sess = tf.Session()
# equal(): tensorflow에서 인수로 지정한 값이 같으면 True, 다르면 False를 리턴한다.
print(sess.run(tf.equal(1, 1)), sess.run(tf.equal(1, 2)))
# equal() 메소드로 예측값과 실제값이 같은가 비교 후 True 또는 False를 cast() 메소드로 1 또는 0으로 캐스팅하고 그 결과의 평균을 낸다.
# 그러면 정확도가 계산되는 것
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.float32))

True False


In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(3001):
    # sess.run([실행할 내용, ...], feed_dict={플레이스홀더 이름: 플레이스홀더에 저장할 데이터, ...})
    a_, b_, loss_, _, predict_, accuracy_ = sess.run([a, b, loss, gradient_descent, predict, accuracy], feed_dict={X: xData, Y: yData})
    if epoch % 300 == 0:
        print('epoch: {:6d}, loss: {:7.4f}, a1: {:7.4f}, a2: {:7.4f}, b: {:7.4f}'.format(epoch, loss_, a_[0][0], a_[1][0], b_[0]))
        # print('epoch: %6d, loss: %7.4f, a: %7.4f, b: %7.4f' % (epoch, sess.run(loss), sess.run(a), sess.run(b)))
print('실제값\n', yData, sep='')
print('예상값\n', predict_, sep='')
print('정확도: {:6.2%}'.format(accuracy_))

epoch:      0, loss:  1.6798, a1:  0.6537, a2: -0.0470, b:  0.2507
epoch:    300, loss:  0.2687, a1:  0.8841, a2: -0.6553, b: -2.4063
epoch:    600, loss:  0.1936, a1:  0.8579, a2: -0.3488, b: -3.8638
epoch:    900, loss:  0.1514, a1:  0.7578, a2: -0.0069, b: -4.9282
epoch:   1200, loss:  0.1238, a1:  0.6484, a2:  0.3081, b: -5.7723
epoch:   1500, loss:  0.1044, a1:  0.5456, a2:  0.5871, b: -6.4730
epoch:   1800, loss:  0.0901, a1:  0.4535, a2:  0.8323, b: -7.0722
epoch:   2100, loss:  0.0792, a1:  0.3720, a2:  1.0482, b: -7.5958
epoch:   2400, loss:  0.0706, a1:  0.3002, a2:  1.2394, b: -8.0609
epoch:   2700, loss:  0.0636, a1:  0.2368, a2:  1.4100, b: -8.4793
epoch:   3000, loss:  0.0579, a1:  0.1805, a2:  1.5632, b: -8.8595
실제값
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
예상값
[[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]]
정확도: 100.00%
